# LDA with Gibbs Sampling

## Import Libraries

In [ ]:
from scipy.special import psi, polygamma, gammaln
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
spacy.load('en_core_web_sm')
from spacy.lang.en import English
import nltk
from nltk.corpus import reuters, wordnet as wn
from nltk.corpus import stopwords
import os
from LDA import LDA_sample

## Data Preprocessing

In [ ]:
stops = stopwords.words("english")
# Add additional stop words
stops += [
    "a", "about", "above", "across", "after", "afterwards", "again", "against",
    "all", "almost", "alone", "along", "already", "also", "although", "always",
    "am", "among", "amongst", "amoungst", "amount", "an", "and", "another",
    "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are",
    "around", "as", "at", "back", "be", "became", "because", "become",
    "becomes", "becoming", "been", "before", "beforehand", "behind", "being",
    "below", "beside", "besides", "between", "beyond", "bill", "both",
    "bottom", "but", "by", "call", "can", "cannot", "cant", "co", "con",
    "could", "couldnt", "cry", "de", "describe", "detail", "do", "done",
    "down", "due", "during", "each", "eg", "eight", "either", "eleven", "else",
    "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone",
    "everything", "everywhere", "except", "few", "fifteen", "fifty", "fill",
    "find", "fire", "first", "five", "for", "former", "formerly", "forty",
    "found", "four", "from", "front", "full", "further", "get", "give", "go",
    "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter",
    "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his",
    "how", "however", "hundred", "i", "ie", "if", "in", "inc", "indeed",
    "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter",
    "latterly", "least", "less", "ltd", "made", "many", "may", "me",
    "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly",
    "move", "much", "must", "my", "myself", "name", "namely", "neither",
    "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone",
    "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on",
    "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our",
    "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps",
    "please", "put", "rather", "re", "same", "see", "seem", "seemed"
]

In [ ]:
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

### Dataset 1: NLTK reuters
    a dataset of new articles, using the titles

In [ ]:
# fetch titles only, 2000 docs only 
dataset = []
vocab = []

i = 0
for file_id in reuters.fileids():
    
    doc = [get_lemma(w.lower()) for w in reuters.words(file_id) \
                 if (w.isupper()) \
                 if (w.lower() not in stops) \
                 and (not w.isnumeric())]
    if doc:
        doc = [t for t in doc if len(t) > 1]
        dataset.append(doc)
        vocab += doc
        i += 1

    if i >= 2000:
        break

dataset = [[token for token in sublist if len(token) > 1] for sublist in dataset]

In [ ]:
print(len(dataset))

In [ ]:
dataset

### Dataset 2: dataset.csv
    a dataset of research paper titles

In [ ]:
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [ ]:
def tokenize_text(text):
    tokens = tokenize(text)
    tokens = [t for t in tokens if len(t) > 4]
    tokens = [t for t in tokens if t not in stops]
    tokens = [get_lemma(t) for t in tokens]
    return tokens

In [ ]:
dataset2 = []
vocab2 = []
with open('dataset.csv') as f:
    for line in f:
        tokened_line = tokenize_text(line)
        vocab2 += tokened_line
        dataset2.append(tokened_line)

In [ ]:
print(len(dataset2))

## Preping data for LDA

### For Dataset 1

In [ ]:
vocab = list(set(vocab))
word_to_ix = {w: i for i, w in enumerate(vocab)}

def seq_to_ix(seq, vocab=vocab):
    # len(vocab), which is the last index, is for the <unk> (unknown) token
    unk_idx = len(vocab)
    return np.array(list(map(lambda w: word_to_ix.get(w, unk_idx), seq)))

data = {
    "data1": list(map(seq_to_ix, dataset))
}
docs = data["data1"]

### For Dataset 2

In [ ]:
vocab2 = list(set(vocab2))
word_to_ix2 = {w: i for i, w in enumerate(vocab2)}

def seq_to_ix(seq, vocab=vocab2):
    # len(vocab), which is the last index, is for the <unk> (unknown) token
    unk_idx = len(vocab)
    return np.array(list(map(lambda w: word_to_ix2.get(w, unk_idx), seq)))

data = {
    "data2": list(map(seq_to_ix, dataset2))
}
docs2 = data["data2"]

## LDA with Gibbs

In [ ]:
lda_news = LDA_sample(docs, vocab, n_topic=10)
lda_news.run_gibbs()

In [ ]:
def init_lda(docs, vocab, n_topic, random_state=0):

    global V, k, N, M, alpha, eta, n_iw, n_di

    np.random.seed(random_state)

    V = len(vocab) # size of vocab
    k = n_topic  # number of topics
    # N = num words in each doc
    N = np.array([doc.shape[0] for doc in docs])
    M = len(docs) #num of docs

    print(f"V: {V}\nk: {k}\nN: {N[:10]}...\nM: {M}")

    # alpha is initialized as a random gamma distribution, which acts as the parameter for the Dirichlet prior for topics
    alpha = np.random.gamma(shape=100, scale=0.01, size=1)  # one for all k
    # eta is similarly initialized for the Dirichlet prior for words
    eta = np.random.gamma(shape=100, scale=0.01, size=1)  # one for all V
    print(f"alpha: {alpha}\n_eta: {eta}")
    
    # n_iw is a matrix for counting the number of times each word is assigned to each topic
    n_iw = np.zeros((k, V), dtype=int)
    # n_di is a matrix for counting the number of words in each document that are assigned to each topic
    n_di = np.zeros((M, k), dtype=int)
    print(f"n_iw: dim {n_iw.shape}\nn_di: dim {n_di.shape}")
    

In [ ]:
def _init_gibbs(docs, vocab, n_topic, n_gibbs=2000):
    """
    Initialize t=0 state for Gibbs sampling.
    Replace initial word-topic assignment ndarray (M, N, N_GIBBS) in-place.
    """
    # initialize variables
    init_lda(docs, vocab, n_topic=n_topic)
    
    # word-topic assignment array (M, N, N_GIBBS)
    global assign
    N_max = max(N)
    assign = np.zeros((M, N_max, n_gibbs+1), dtype=int)
    print(f"assign: dim {assign.shape}")
    
    # initial assignment
    for d in range(M):
        for n in range(N[d]):
            # randomly assign topic to word w_{dn}
            w_dn = docs[d][n]
            assign[d, n, 0] = np.random.randint(k)

            # increment counters
            i = assign[d, n, 0]
            n_iw[i, w_dn] += 1
            n_di[d, i] += 1

In [ ]:
def _conditional_prob(w_dn, d):
    """
    P(z_{dn}^i=1 | z_{(-dn)}, w)
    """
    prob = np.empty(k)
    
    for i in range(k):
        # P(w_dn | z_i)
        _1 = (n_iw[i, w_dn] + eta) / (n_iw[i, :].sum() + V*eta)
        # P(z_i | d)
        _2 = (n_di[d, i] + alpha) / (n_di[d, :].sum() + k*alpha)
        
        prob[i] = _1 * _2
    
    return prob / prob.sum()

In [ ]:
def run_gibbs(docs, vocab, n_topic, n_gibbs=2000):
    """
    Run collapsed Gibbs sampling
    """
    # initialize required variables
    _init_gibbs(docs, vocab, n_topic, n_gibbs)
    
    print("\n", "="*10, "START SAMPLER", "="*10)
    
    # run the sampler
    for t in range(n_gibbs):
        for d in range(M):
            for n in range(N[d]):
                w_dn = docs[d][n]
                
                # decrement counters
                i_t = assign[d, n, t]  # previous assignment
                n_iw[i_t, w_dn] -= 1
                n_di[d, i_t] -= 1

                # assign new topics
                prob = _conditional_prob(w_dn, d)
                i_tp1 = np.argmax(np.random.multinomial(1, prob))

                # increment counter according to new assignment
                n_iw[i_tp1, w_dn] += 1
                n_di[d, i_tp1] += 1
                assign[d, n, t+1] = i_tp1
        
        # print out status
        if ((t+1) % 50 == 0):
            print(f"Sampled {t+1}/{n_gibbs}")

### Run Gibbs for Dataset 1

In [ ]:
run_gibbs(docs, vocab, n_topic=10, n_gibbs=2000)

In [ ]:
beta = np.empty((k,V))
theta = np.empty((M, k))

for j in range(V):
    for i in range(k):
        beta[i, j] = (n_iw[i, j] + eta) / (n_iw[i, :].sum() + V*eta)

for d in range(M):
    for i in range(k):
        theta[d, i] = (n_di[d, i] + alpha) / (n_di[d, :].sum() + k*alpha)

### Run Gibbs for Dataset 2

In [ ]:
run_gibbs(docs2, vocab2, n_topic=10, n_gibbs=2000)

In [ ]:
beta2 = np.empty((k,V))
theta2 = np.empty((M, k))

for j in range(V):
    for i in range(k):
        beta2[i, j] = (n_iw[i, j] + eta) / (n_iw[i, :].sum() + V*eta)

for d in range(M):
    for i in range(k):
        theta2[d, i] = (n_di[d, i] + alpha) / (n_di[d, :].sum() + k*alpha)

## Evaulation

In [ ]:
def n_most_important(beta_i,vocab,n):
    """
    find the index of the largest `n` values in a list
    """
    
    max_values = beta_i.argsort()[-n:][::-1]
    return np.array(vocab)[max_values]

### Dataset 1

In [ ]:
for i in range(k):
    print(f"TOPIC {i:02}: {n_most_important(beta[i],vocab,9)}")

### Dataset 2

In [ ]:
for i in range(k):
    print(f"TOPIC {i:02}: {n_most_important(beta2[i],vocab2,9)}")